In [1]:
from IPython.display import HTML
HTML(open('../style.css', 'r').read())

# Sudoku

In [2]:
%%capture
%run 07-Davis-Putnam-JW.ipynb

The finish mathematician Arto Inkala claims to have created the [hardest sudoku](https://abcnews.go.com/blogs/headlines/2012/06/can-you-solve-the-hardest-ever-sudoku) ever.  It is defined below.

In [3]:
def create_puzzle():
    return [ [ 8 , "*", "*", "*", "*", "*", "*", "*", "*"], 
             ["*", "*",  3 ,  6 , "*", "*", "*", "*", "*"],
             ["*",  7 , "*", "*",  9 , "*",  2 , "*", "*"],
             ["*",  5 , "*", "*", "*",  7 , "*", "*", "*"],
             ["*", "*", "*", "*",  4 ,  5 ,  7 , "*", "*"],
             ["*", "*", "*",  1 , "*", "*", "*",  3 , "*"],
             ["*", "*",  1 , "*", "*", "*", "*",  6 ,  8 ],
             ["*", "*",  8 ,  5 , "*", "*", "*",  1 , "*"],
             ["*",  9 , "*", "*", "*", "*",  4 , "*", "*"]
           ]

We will solve this Sudoku using the Davis-Putnam algorithm.  We use the following variables:
* `Q<r,c,d>` is a Boolean variable stating that the field in row `r` and column `c` holds the digit `d`.
  Here, `r`, `c`, `d` are all elements from the set $\{1,\cdots,9\}$.
    
The function `var(row, col, digit)` returns a formated string that is interpreted as a variable name.

In [4]:
def var(row, col, digit):
    return f'Q<{row},{col},{digit}>'

In [5]:
var(1,2,3)

'Q<1,2,3>'

The function `atMostOne(S)` takes a set `S` of propositional variables as its argument.  It returns a set of clauses
expressing the fact that at most one of the variables of `S` is true.

In [6]:
def atMostOne(S): 
    return { frozenset({('¬', p), ('¬', q)}) for p in S
                                             for q in S 
                                             if  p < q 
           }

The function `atLeastOne(S)` takes a set `S` of propositional variables as its argument.  It returns a set of clauses
expressing the fact that at least one of the variables of `S` is true.

In [7]:
def atLeastOne(S):
    return { frozenset(S) }

The function `exactlyOne(S)` takes a set `S` of propositional variables as its argument.  It returns a set of clauses
expressing the fact that exactly one of the variables of `S` is true.

In [8]:
def exactlyOne(S):
    return atMostOne(S) | atLeastOne(S)

In [9]:
exactlyOne({'a', 'b', 'c'})

{frozenset({('¬', 'a'), ('¬', 'c')}),
 frozenset({'a', 'b', 'c'}),
 frozenset({('¬', 'a'), ('¬', 'b')}),
 frozenset({('¬', 'b'), ('¬', 'c')})}

The function `allDifferent` takes a list `L` of pairs of indices as its argument.  The elements of `L` are pairs of the form
`(row, col)`, where both `row` and `col` are elements of the set $\{1, \cdots, 9\}$.
It returns a set of formulas expressing that all Sudoku fields specified by the coordinate pairs in `L` take different digits as values.

In [10]:
def allDifferent(L):
    Clauses = set()
    for digit in range(1, 10):
        Clauses |= exactlyOne({ var(col, row, digit)  for col, row in L })
    return Clauses                                                         

In [11]:
allDifferent([(1,c) for c in range(1,10)])

{frozenset({('¬', 'Q<1,1,5>'), ('¬', 'Q<1,3,5>')}),
 frozenset({'Q<1,1,2>',
            'Q<1,2,2>',
            'Q<1,3,2>',
            'Q<1,4,2>',
            'Q<1,5,2>',
            'Q<1,6,2>',
            'Q<1,7,2>',
            'Q<1,8,2>',
            'Q<1,9,2>'}),
 frozenset({('¬', 'Q<1,1,1>'), ('¬', 'Q<1,6,1>')}),
 frozenset({('¬', 'Q<1,7,1>'), ('¬', 'Q<1,8,1>')}),
 frozenset({('¬', 'Q<1,4,3>'), ('¬', 'Q<1,6,3>')}),
 frozenset({('¬', 'Q<1,4,3>'), ('¬', 'Q<1,8,3>')}),
 frozenset({('¬', 'Q<1,3,3>'), ('¬', 'Q<1,9,3>')}),
 frozenset({('¬', 'Q<1,8,5>'), ('¬', 'Q<1,9,5>')}),
 frozenset({('¬', 'Q<1,5,6>'), ('¬', 'Q<1,6,6>')}),
 frozenset({('¬', 'Q<1,8,7>'), ('¬', 'Q<1,9,7>')}),
 frozenset({('¬', 'Q<1,1,3>'), ('¬', 'Q<1,5,3>')}),
 frozenset({('¬', 'Q<1,2,1>'), ('¬', 'Q<1,7,1>')}),
 frozenset({('¬', 'Q<1,6,2>'), ('¬', 'Q<1,7,2>')}),
 frozenset({('¬', 'Q<1,6,4>'), ('¬', 'Q<1,7,4>')}),
 frozenset({('¬', 'Q<1,2,5>'), ('¬', 'Q<1,7,5>')}),
 frozenset({('¬', 'Q<1,2,4>'), ('¬', 'Q<1,7,4>')}),
 f

The function `exactlyOneDigit(row, col)` takes integers `row` and `col` as arguments.  These specify the row and column of a field in a Sudoku.  The function returns a set of clauses specifying that exactly one of the variables

* `Q<row,col,1>`, `Q<row,col,2>`, $\cdots$, `Q<row,col,9>`

is `True`.

In [12]:
def exactlyOneDigit(row, col):
    return exactlyOne({ var(row, col, digit) for digit in range(1, 10) })

In [13]:
exactlyOneDigit(1, 1)

{frozenset({('¬', 'Q<1,1,2>'), ('¬', 'Q<1,1,6>')}),
 frozenset({('¬', 'Q<1,1,4>'), ('¬', 'Q<1,1,6>')}),
 frozenset({('¬', 'Q<1,1,3>'), ('¬', 'Q<1,1,9>')}),
 frozenset({('¬', 'Q<1,1,4>'), ('¬', 'Q<1,1,8>')}),
 frozenset({('¬', 'Q<1,1,1>'), ('¬', 'Q<1,1,5>')}),
 frozenset({('¬', 'Q<1,1,3>'), ('¬', 'Q<1,1,5>')}),
 frozenset({('¬', 'Q<1,1,6>'), ('¬', 'Q<1,1,7>')}),
 frozenset({('¬', 'Q<1,1,2>'), ('¬', 'Q<1,1,8>')}),
 frozenset({('¬', 'Q<1,1,3>'), ('¬', 'Q<1,1,7>')}),
 frozenset({('¬', 'Q<1,1,1>'), ('¬', 'Q<1,1,3>')}),
 frozenset({('¬', 'Q<1,1,2>'), ('¬', 'Q<1,1,9>')}),
 frozenset({('¬', 'Q<1,1,5>'), ('¬', 'Q<1,1,9>')}),
 frozenset({('¬', 'Q<1,1,8>'), ('¬', 'Q<1,1,9>')}),
 frozenset({('¬', 'Q<1,1,1>'), ('¬', 'Q<1,1,2>')}),
 frozenset({('¬', 'Q<1,1,6>'), ('¬', 'Q<1,1,8>')}),
 frozenset({('¬', 'Q<1,1,2>'), ('¬', 'Q<1,1,5>')}),
 frozenset({('¬', 'Q<1,1,3>'), ('¬', 'Q<1,1,8>')}),
 frozenset({('¬', 'Q<1,1,1>'), ('¬', 'Q<1,1,8>')}),
 frozenset({('¬', 'Q<1,1,4>'), ('¬', 'Q<1,1,5>')}),
 frozenset({

The function `constraints_from_puzzle`  returns a set of clauses stating that the variables corresponding to numbers that are already given in the Sudoku puzzle take the values that are specified.

In [14]:
def constraints_from_puzzle():
    Puzzle = create_puzzle()
    Variables = [ var(row + 1, col + 1, Puzzle[row][col]) for row in range(9)
                                                          for col in range(9)
                                                          if  Puzzle[row][col] != '*'
                ]
    return { frozenset({ var }) for var in Variables }

In [15]:
constraints_from_puzzle()

{frozenset({'Q<5,6,5>'}),
 frozenset({'Q<8,8,1>'}),
 frozenset({'Q<7,8,6>'}),
 frozenset({'Q<3,7,2>'}),
 frozenset({'Q<6,4,1>'}),
 frozenset({'Q<6,8,3>'}),
 frozenset({'Q<9,7,4>'}),
 frozenset({'Q<7,3,1>'}),
 frozenset({'Q<2,3,3>'}),
 frozenset({'Q<5,7,7>'}),
 frozenset({'Q<8,3,8>'}),
 frozenset({'Q<3,2,7>'}),
 frozenset({'Q<8,4,5>'}),
 frozenset({'Q<3,5,9>'}),
 frozenset({'Q<7,9,8>'}),
 frozenset({'Q<1,1,8>'}),
 frozenset({'Q<5,5,4>'}),
 frozenset({'Q<4,6,7>'}),
 frozenset({'Q<9,2,9>'}),
 frozenset({'Q<4,2,5>'}),
 frozenset({'Q<2,4,6>'})}

The function `all_constraints` returns a CSP that encodes the given sudoku as a CSP.

In [16]:
def all_constraints(): 
    L = [1, 2, 3, 4, 5, 6, 7, 8, 9]
    # the constraints from the puzzle have to be satisfied
    Clauses = constraints_from_puzzle()
    # there is exactly one digit in every field
    for row in L:
        for col in L:
            Clauses |= exactlyOneDigit(row, col) 
    # all entries in a row are unique
    for row in L:
        Clauses |= allDifferent([ (row, col) for col in L ]) 
    # all entries in a column are unique
    for col in L:
        Clauses |= allDifferent([ (row, col) for row in L ])
    # all entries in a 3x3 square are unique    
    for r in range(3):
        for c in range(3):
            S = [ (r * 3 + row, c * 3 + col) for row in range(1, 4)
                                             for col in range(1, 4) 
                ]
            Clauses |= allDifferent(S)
    return Clauses

In [17]:
for Clause in all_constraints():
    if len(Clause) == 1:
        print(set(Clause))
for Clause in all_constraints():
    if len(Clause) == 9:
        print(set(Clause))

{'Q<8,3,8>'}
{'Q<6,8,3>'}
{'Q<4,6,7>'}
{'Q<3,7,2>'}
{'Q<7,9,8>'}
{'Q<4,2,5>'}
{'Q<5,5,4>'}
{'Q<1,1,8>'}
{'Q<7,3,1>'}
{'Q<5,6,5>'}
{'Q<9,2,9>'}
{'Q<9,7,4>'}
{'Q<2,4,6>'}
{'Q<6,4,1>'}
{'Q<5,7,7>'}
{'Q<8,4,5>'}
{'Q<2,3,3>'}
{'Q<3,2,7>'}
{'Q<3,5,9>'}
{'Q<7,8,6>'}
{'Q<8,8,1>'}
{'Q<6,8,8>', 'Q<4,8,8>', 'Q<5,7,8>', 'Q<6,7,8>', 'Q<6,9,8>', 'Q<5,8,8>', 'Q<4,9,8>', 'Q<4,7,8>', 'Q<5,9,8>'}
{'Q<4,4,4>', 'Q<7,4,4>', 'Q<5,4,4>', 'Q<1,4,4>', 'Q<8,4,4>', 'Q<2,4,4>', 'Q<6,4,4>', 'Q<3,4,4>', 'Q<9,4,4>'}
{'Q<6,7,5>', 'Q<1,7,5>', 'Q<8,7,5>', 'Q<4,7,5>', 'Q<5,7,5>', 'Q<3,7,5>', 'Q<2,7,5>', 'Q<7,7,5>', 'Q<9,7,5>'}
{'Q<3,7,1>', 'Q<2,8,1>', 'Q<3,9,1>', 'Q<2,7,1>', 'Q<1,7,1>', 'Q<3,8,1>', 'Q<2,9,1>', 'Q<1,9,1>', 'Q<1,8,1>'}
{'Q<5,1,1>', 'Q<6,2,1>', 'Q<4,2,1>', 'Q<4,1,1>', 'Q<6,3,1>', 'Q<5,3,1>', 'Q<5,2,1>', 'Q<4,3,1>', 'Q<6,1,1>'}
{'Q<5,5,9>', 'Q<2,5,9>', 'Q<6,5,9>', 'Q<7,5,9>', 'Q<4,5,9>', 'Q<1,5,9>', 'Q<8,5,9>', 'Q<3,5,9>', 'Q<9,5,9>'}
{'Q<9,1,5>', 'Q<9,2,5>', 'Q<8,3,5>', 'Q<8,2,5>', 'Q<9,3,5>', 'Q<7,2,5>', 

In [18]:
len(all_constraints())

10551

The function `solve(Constraints, Variables)` receives two arguments:
- `Constraints` is a set of formulas representing a constraint satisfaction problem.
- `Variables`   is the set of variables that occur in this formulas.

The function computes a solution to the given problem and returns this solution.

In [19]:
def sudoku():
    Clauses  = all_constraints()
    Solution = solve(Clauses)
    if Solution != { frozenset() }:
        return Solution
    else:
        print(f'The problem is not solvable!')
        return None

In [ ]:
%%time
Solution = sudoku()

## Graphical Representation

The following line needs to be executed once to install the package `problem_visuals`.

In [ ]:
!pip install git+https://github.com/reclinarka/problem_visuals

In [ ]:
from problem_visuals.games.sudoku.grid import Grid

In [ ]:
def transform_solution(Solution):
    Result = {}
    for UnitClause in Solution:
        literal = arb(UnitClause)
        if isinstance(literal, str):
            row   = int(literal[2:3])
            col   = int(literal[4:5])
            digit = int(literal[6:7])
            Result[f'V{row}{col}'] = digit
    return Result

In [ ]:
def show_solution(Solution, width='50%'):
    Solution = transform_solution(Solution)
    Sudoku = create_puzzle()
    for row in range(9):
        for col in range(9):
            if Sudoku[row][col] != '*':
                del Solution[f'V{row+1}{col+1}']
    return Grid(state=Sudoku, assigned=Solution, html_width=width)

In [ ]:
show_solution(Solution)